<a href="https://colab.research.google.com/github/jigmam/reactllm/blob/main/convertidor_de_modelos_funciona_conTPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U keras-nlp
!pip install -q -U keras>=3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 570.5/570.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 15.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.3.3 which is incompatible.


In [ ]:
import keras
import keras_nlp
import os
import tensorflow as tf

In [ ]:
saved_model_dir = '/content/drive/MyDrive/modelos/gemma'

In [ ]:
loaded_model = keras_nlp.models.GemmaTokenizer.from_preset(saved_model_dir,dtype="bfloat16", load_weights=True )

In [ ]:
# Obtener la configuración del tokenizador en formato JSON
config_json = loaded_model.get_config()

# Guardar la configuración en un archivo .model
with open('/content/drive/MyDrive/modelos/tokenizer2.model', 'w') as model_file:
     model_file.write(str(config_json))

In [ ]:
import tensorflow as tf

In [ ]:
model = loaded_model.backbone
print("<======= EXPORTING =====>")

export_archive = keras.export.ExportArchive()
print("<======= TRACKING =====>")

export_archive.track(model)
export_archive.add_endpoint(
    name="serve",
    fn=model.call,
    input_signature=[{
        "token_ids": tf.TensorSpec(shape=(None, 3), dtype=tf.float32),
        "padding_mask": tf.TensorSpec(shape=(None, 3), dtype=tf.float32),
    }],
)

In [ ]:
export_archive.write_out('gemma_2')

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model('gemma_2/')
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Save the model
with open('/content/drive/MyDrive/modelos/model-quantize.tflite', 'wb') as f:
    f.write(tflite_model)